# Sniffing packets
- sniff() function allows us to capture filtered packets of interest
- equivalent to tcpdump and wireshark
- also has built-in wireshark function to parse big and complex packet captures

In [29]:
from scapy.all import *
conf.verb=1
conf.color_theme = RastaTheme()

In [ ]:
help(sniff)

In [22]:
# count argument value will sniff that many packets
pkts = sniff(count=2)

In [23]:
pkts

<Sniffed: TCP:0 UDP:2 ICMP:0 Other:0>

In [24]:
pkts.summary()

Ether / IP / UDP / DNS Qry "b'scapy.readthedocs.io.'" 
Ether / IP / UDP / DNS Ans 


## Sniffing ping/icmp packets 
### Generate icmp packets using ping
1. open a terminal and run $ arping [ip of another vm/gateway]
2. sniff the packets using scapy

In [25]:
# note: run ping google.com on a terminal to generate icmp packets
pkts = sniff(count=2, filter="icmp")

In [26]:
pkts

<Sniffed: TCP:0 UDP:0 ICMP:2 Other:0>

In [27]:
pkts.summary()

Ether / IP / ICMP 192.168.1.145 > 192.168.1.139 echo-request 0 / Raw / Padding
Ether / IP / ICMP 192.168.1.139 > 192.168.1.145 echo-reply 0 / Raw


In [30]:
pkts.show()

0000 Ether / IP / ICMP 192.168.1.145 > 192.168.1.139 echo-request 0 / Raw / Padding
0001 Ether / IP / ICMP 192.168.1.139 > 192.168.1.145 echo-reply 0 / Raw


In [31]:
pkts[0].show()

###[ Ethernet ]### 
  dst= 08:00:27:f7:e8:f7
  src= e4:9a:dc:dc:e7:0c
  type= IPv4
###[ IP ]### 
     version= 4
     ihl= 5
     tos= 0x0
     len= 44
     id= 17854
     flags= 
     frag= 0
     ttl= 64
     proto= icmp
     chksum= 0xb0a6
     src= 192.168.1.145
     dst= 192.168.1.139
     \options\
###[ ICMP ]### 
        type= echo-request
        code= 0
        chksum= 0x97a1
        id= 0x8c29
        seq= 0x3c18
###[ Raw ]### 
           load= '\t\x9e$\x0cd2$6\x90\xc7\xe1\x94\xc2H\xadd'
###[ Padding ]### 
              load= '\x00\x00'



In [32]:
pkts[1].show()

###[ Ethernet ]### 
  dst= e4:9a:dc:dc:e7:0c
  src= 08:00:27:f7:e8:f7
  type= IPv4
###[ IP ]### 
     version= 4
     ihl= 5
     tos= 0x0
     len= 44
     id= 4283
     flags= 
     frag= 0
     ttl= 64
     proto= icmp
     chksum= 0xe5a9
     src= 192.168.1.139
     dst= 192.168.1.145
     \options\
###[ ICMP ]### 
        type= echo-reply
        code= 0
        chksum= 0x9fa1
        id= 0x8c29
        seq= 0x3c18
###[ Raw ]### 
           load= '\t\x9e$\x0cd2$6\x90\xc7\xe1\x94\xc2H\xadd'



In [ ]:
hexdump(pkts[0])

## Sniffing arp packets 
### Generate arp packets using arping command
1. open a terminal and run $ arping [ip of another vm/gateway]
2. sniff the packets using scapy

In [33]:
# ping a non-existant private ip or gateway or use arping
pkts = sniff(count=5, filter="arp")

In [34]:
pkts.summary()

Ether / ARP is at c8:b3:73:1f:95:b4 says 192.168.1.1 / Padding
Ether / ARP who has 192.168.1.139 says 192.168.1.1 / Padding
Ether / ARP is at 08:00:27:f7:e8:f7 says 192.168.1.139
Ether / ARP who has 192.168.1.1 says 192.168.1.139
Ether / ARP is at c8:b3:73:1f:95:b4 says 192.168.1.1 / Padding


In [ ]:
pkts[0].show()

In [ ]:
pkts[1].show()

In [ ]:
pkts[0].command() # show command to generate the first packet

## Custom formatted ARP Monitor
- https://thepacketgeek.com/scapy-sniffing-with-custom-actions-part-1/
- run the following code and ping a valid local ip from a terminal to generate 

In [35]:
# check ARP table
! arp -n

Address                  HWtype  HWaddress           Flags Mask            Iface
192.168.1.10                     (incomplete)                              eth0
192.168.1.145            ether   e4:9a:dc:dc:e7:0c   C                     eth0
192.168.1.100                    (incomplete)                              eth0
192.168.1.3                      (incomplete)                              eth0
192.168.1.2                      (incomplete)                              eth0
192.168.1.1              ether   c8:b3:73:1f:95:b4   C                     eth0


In [36]:
# clear arp cache and validate
! ip -s -s neigh flush all

192.168.1.10 dev eth0  used 85/151/82 probes 6 FAILED
192.168.1.145 dev eth0 lladdr e4:9a:dc:dc:e7:0c used 103/102/63 probes 1 STALE
192.168.1.100 dev eth0  used 79/141/78 probes 6 FAILED
192.168.1.3 dev eth0  used 95/161/94 probes 6 FAILED
192.168.1.2 dev eth0  used 104/168/101 probes 6 FAILED
192.168.1.1 dev eth0 lladdr c8:b3:73:1f:95:b4 ref 1 used 42/37/37 probes 1 REACHABLE

*** Round 1, deleting 6 entries ***
*** Flush is complete after 1 round ***


In [37]:
# validate
! arp -n

In [ ]:
from collections import Counter
from scapy.all import ARP, sniff

def arp_display(pkt):
    if pkt[ARP].op == 1: #who-has (request)
        return f"Request: {pkt[ARP].psrc} is asking about {pkt[ARP].pdst}"
    if pkt[ARP].op == 2: #is-at (response)
        return f"*Response: {pkt[ARP].hwsrc} has address {pkt[ARP].psrc}"

sniff(prn=arp_display, filter="arp", store=0)

Request: 192.168.1.139 is asking about 192.168.1.145
Request: 192.168.1.139 is asking about 192.168.1.145
Request: 192.168.1.139 is asking about 192.168.1.145
Request: 192.168.1.139 is asking about 192.168.1.145
Request: 192.168.1.139 is asking about 192.168.1.145
Request: 192.168.1.139 is asking about 192.168.1.145
Request: 192.168.1.139 is asking about 192.168.1.145
Request: 192.168.1.139 is asking about 192.168.1.145
*Response: e4:9a:dc:dc:e7:0c has address 192.168.1.145
Request: 192.168.1.1 is asking about 192.168.1.102
*Response: c8:b3:73:1f:95:b4 has address 192.168.1.1
Request: 192.168.1.1 is asking about 192.168.1.102
Request: 192.168.1.1 is asking about 192.168.1.102
Request: 192.168.1.1 is asking about 192.168.1.102
Request: 192.168.1.1 is asking about 192.168.1.102


In [ ]:
help(sniff)

## Keeping track of number of packets sniffed

In [ ]:
from collections import Counter
from scapy.all import sniff

## Create a Packet Counter
packet_counts = Counter()

## Define our Custom Action function
def custom_action(packet):
    # Create tuple of Src/Dst in sorted order
    key = tuple(sorted([packet[0][1].src, packet[0][1].dst]))
    packet_counts.update([key])
    return f"Packet #{sum(packet_counts.values())}: {packet[0][1].src} ==> {packet[0][1].dst}"

## Setup sniff, filtering for IP traffic
sniff(filter="ip", prn=custom_action, count=10)

## Print out packet count per A <--> Z address pair
print("\n".join(f"{f'{key[0]} <--> {key[1]}'}: {count}" for key, count in packet_counts.items()))

## Importing and Exporting Packets
## PCAP format
- save capture packets to pcap file for use at later time or with different applications

In [ ]:
# note: run ping google.com on a terminal to generate icmp packets
pkts = sniff(count=5, filter="icmp")

In [ ]:
from scapy.all import wrpcap
wrpcap('icmp.pcap', pkts)

In [ ]:
# use scapy's hexdump to see pkts
from scapy.all import hexdump
hexdump(pkts[0])

In [ ]:
# use bash hexdump
! hexdump -C icmp.pcap

In [ ]:
# restore previously saved pcap file
from scapy.all import rdpcap
pkts1 = rdpcap('icmp.pcap')

In [ ]:
pkts1.summary()

In [ ]:
pkts1.show()

In [ ]:
new_pkt = pkts[0]

In [ ]:
new_pkt

## open icmp.pcap file in wireshark to analyze